# Week 2 Examples — Architectures, TCP Sockets, and Framing

This notebook demonstrates a localhost TCP echo/chat server with safe message framing (length-prefixed) and basic metrics. It also includes a quick architecture recap and capacity estimate example.

Requirements: Python 3.9+, localhost networking enabled.

---

## Architecture recap (quick)
- Client-Server: single tier to scale first; simple operability
- Layered (N-tier): separate API/business/data; independent scaling
- Peer-to-Peer: resilience and locality; harder coordination/consistency

Capacity back-of-the-envelope:
- rps_per_node ≈ 1 / avg_service_time
- cluster_rps ≈ nodes * rps_per_node * utilization

---

## TCP is a byte stream: implement length-prefixed framing
We send: 4-byte big-endian length followed by UTF-8 payload.

Utility functions:
- `send_msg(sock, text)`
- `recv_msg(sock)`


In [ ]:
import socket, struct, threading, time
from contextlib import closing


def send_msg(sock: socket.socket, text: str):
    data = text.encode('utf-8')
    header = struct.pack('!I', len(data))
    sock.sendall(header + data)


def recv_all(sock: socket.socket, n: int) -> bytes:
    buf = bytearray()
    while len(buf) < n:
        chunk = sock.recv(n - len(buf))
        if not chunk:
            raise ConnectionError('socket closed')
        buf.extend(chunk)
    return bytes(buf)


def recv_msg(sock: socket.socket) -> str:
    header = recv_all(sock, 4)
    (length,) = struct.unpack('!I', header)
    data = recv_all(sock, length)
    return data.decode('utf-8')


## Echo/Chat server (localhost)
- Accepts multiple clients via threads
- Echoes each message to sender (echo) and broadcasts to others (chat)
- Tracks simple metrics: messages/sec and average service time


In [ ]:
class ChatServer:
    def __init__(self, host='127.0.0.1', port=50555):
        self.host = host
        self.port = port
        self.sock = None
        self.clients = set()
        self.lock = threading.Lock()
        self.running = threading.Event()
        self.msg_count = 0
        self.total_service_time = 0.0

    def start(self):
        self.running.set()
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind((self.host, self.port))
        self.sock.listen(50)
        threading.Thread(target=self._accept_loop, daemon=True).start()
        threading.Thread(target=self._print_metrics_loop, daemon=True).start()

    def _accept_loop(self):
        while self.running.is_set():
            try:
                client, addr = self.sock.accept()
            except OSError:
                break
            with self.lock:
                self.clients.add(client)
            t = threading.Thread(target=self._client_loop, args=(client,), daemon=True)
            t.start()

    def _client_loop(self, client):
        try:
            while self.running.is_set():
                t0 = time.perf_counter()
                msg = recv_msg(client)
                # Echo back
                send_msg(client, f"echo: {msg}")
                # Broadcast to others
                with self.lock:
                    peers = [c for c in self.clients if c is not client]
                for c in peers:
                    try:
                        send_msg(c, f"peer: {msg}")
                    except Exception:
                        pass
                dt = time.perf_counter() - t0
                self.msg_count += 1
                self.total_service_time += dt
        except Exception:
            pass
        finally:
            with self.lock:
                self.clients.discard(client)
            try:
                client.close()
            except Exception:
                pass

    def _print_metrics_loop(self):
        last = time.time()
        last_count = 0
        while self.running.is_set():
            time.sleep(2)
            now = time.time()
            elapsed = now - last
            count = self.msg_count
            delta = count - last_count
            rps = delta/elapsed if elapsed > 0 else 0
            avg_ms = (self.total_service_time/count)*1e3 if count else 0
            print({"messages_total": count, "rps": round(rps,2), "avg_service_ms": round(avg_ms,3)})
            last = now
            last_count = count

    def stop(self):
        self.running.clear()
        if self.sock:
            try:
                self.sock.close()
            except Exception:
                pass


## Client helper
Connects, sends a few messages, receives echos.


In [ ]:
def demo_client(messages=("hello", "world", "from", "client"), host='127.0.0.1', port=50555):
    with closing(socket.create_connection((host, port), timeout=3)) as s:
        s.settimeout(3)
        for m in messages:
            send_msg(s, m)
            echo = recv_msg(s)
            print('recv:', echo)


## Demo run
Start server, run two clients in parallel threads, then stop server.


In [ ]:
server = ChatServer()
server.start()

t1 = threading.Thread(target=demo_client, args=(("a","b","c","d"),), daemon=True)
t2 = threading.Thread(target=demo_client, args=(("x","y","z"),), daemon=True)

t1.start(); t2.start()
t1.join(); t2.join()

# Give metrics thread a moment
time.sleep(3)
server.stop()


## Exercises
1) Modify framing to include a message type byte before the length (simple protocol evolution).\
2) Add timestamps to messages and compute end-to-end latency.\
3) Stress test with 50 concurrent clients; record rps and avg service ms.
